# Life of a Transaction in LUSID

This notebook provides a reference implementation to the [life of a transaction](https://www.finbourne.com/blog/plotting-the-future-of-hedge-fund-technology/life-of-a-transaction) demonstrating the process of loading transactions into LUSID.

While we use python to interact with the LUSID API here, the models and methods are also available through our other maintained SDKs in [C#](https://github.com/finbourne/lusid-sdk-csharp), [Java](https://github.com/finbourne/lusid-sdk-java) and [JavaScript](https://github.com/finbourne/lusid-sdk-js). 

In [1]:
import os
import lusid
import json
from lusid import models
import pandas as pd
from lusid.utilities import ApiClientFactory
from pathlib import Path
from lusidjam import RefreshingToken
import pickle
import pprint
from lusidtools.cocoon.utilities import create_scope_id


# Authenticate our user and create our API client
secrets_path = os.path.join(os.pardir, "secrets.json")
                            
if os.path.exists(secrets_path):
    api_factory = lusid.utilities.ApiClientFactory(api_secrets_filename=secrets_path)
else:
    api_factory = lusid.utilities.ApiClientFactory(
        token=RefreshingToken(),
        app_name="LusidJupyterNotebook")
    
    
print ('LUSID Environment Initialised')
print ('LUSID SDK Version: ', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID SDK Version:  0.5.4202.0


In [2]:
# Build the APIs we are going to use
TransactionPortfoliosApi = api_factory.build(lusid.api.TransactionPortfoliosApi)
PortfoliosApi = api_factory.build(lusid.api.PortfoliosApi)
PropertyDefinitionsApi = api_factory.build(lusid.api.PropertyDefinitionsApi)
InstrumentsApi  = api_factory.build(lusid.api.InstrumentsApi)
SystemConfigurationApi  = api_factory.build(lusid.api.SystemConfigurationApi)

## Load Data files

In [3]:
txns = pd.read_csv("data/transactions-life-of.csv")
txns

,txn_id,trade_date,transaction_type,instrument_desc,client_internal,instrument_currency,instrument_id,quantity,price,net_money,portfolio,base_currency,portfolio_manager_name,Broker
0,tx_00001,2020-01-01T00:00:00.0000000+00:00,FundsIn,CASH_GBP,cash,GBP,Cash in GBP,1000000,1.00,1000000,UK_EQUITY,GBP,John Smith,NaN
1,tx_00002,2020-01-01T00:00:00.0000000+00:00,Buy,Tesco,EQ_001,GBP,TSCO,100000,2.56,256000,UK_EQUITY,GBP,John Smith,NaN
2,tx_00003,2020-01-02T00:00:00.0000000+00:00,Buy,Morrisons,EQ_002,GBP,MRW,120000,1.92,230400,UK_EQUITY,GBP,John Smith,NaN
3,tx_00004,2020-01-02T00:00:00.0000000+00:00,Buy,Sainsbury's,EQ_003,GBP,SBRY,200000,2.26,452000,UK_EQUITY,GBP,John Smith,NaN
4,tx_00005,2020-01-03T00:00:00.0000000+00:00,Buy,Tesco,EQ_001,GBP,TSCO,20000,2.60,52000,UK_EQUITY,GBP,John Smith,NaN


# Document section 4: Load Portfolios

We must begin by specifying what scope are are going to work in. We will also define the portfolio code here. 

In [4]:
scope = "life_of_a_txn-" + create_scope_id()
portfolio_code = "UK_EQUITY"

print(scope)
print(portfolio_code)

life_of_a_txn-3854-a038-167a-56
UK_EQUITY


In [5]:
# Call LUSID to create a property
property_response = PropertyDefinitionsApi.create_property_definition(
definition=models.CreatePropertyDefinitionRequest(
        domain="Portfolio",
        scope=scope,
        code="portfolio_manager_name",
        value_required=False,
        display_name="portfolio_manager_name",
        data_type_id=models.ResourceId(
            scope='system',
            code='string'
        )
    )
)

print(f"Created new property {property_response.key}")

Created new property Portfolio/life_of_a_txn-3854-a038-167a-56/portfolio_manager_name


### Create Portfolio

In [6]:
# Create portfolio with properties
subholding_key={}
created_date="2010-01-01T00:00:00.000000+00:00"

# create request body
portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name="Life of a transaction",
    code = portfolio_code,
    base_currency="GBP",
    created=created_date,
    properties=
        {
        f"Portfolio/{scope}/portfolio_manager_name" : models.ModelProperty(
                                                            key=f"Portfolio/{scope}/portfolio_manager_name",
                                                            value=models.PropertyValue(
                                                                    label_value="Active"
                                                                )
                                                            )
        },
)

# Upload new portfolio to LUSID
response = TransactionPortfoliosApi.create_portfolio(
    scope=scope,
    transaction_portfolio=portfolio_request
)

created = response.version.effective_from
print(f"portfolio '{response.id.code}', in scope {scope} created effective from: "
      f"{created.year}/"
      f"{created.month}/"
      f"{created.day}"
     )

portfolio 'UK_EQUITY', in scope life_of_a_txn-3854-a038-167a-56 created effective from: 2010/1/1


# Document Section 5: Load Instruments

In [7]:
# Call LUSID to create a property
property_response = PropertyDefinitionsApi.create_property_definition(
definition=models.CreatePropertyDefinitionRequest(
            domain="Instrument",
            scope=scope,
            code="portfolio_manager_name",
            value_required=False,
            display_name="portfolio_manager_name",
            data_type_id=models.ResourceId(
                scope='system',
                code='string'
            )
        )
    )

print(f"Created new property {property_response.key}")

Created new property Instrument/life_of_a_txn-3854-a038-167a-56/portfolio_manager_name


Each instrument definition needs to be given an identifier. For equities this could be an Isin or Figi, alternatively for cash this must be "Currency".

In [8]:
# remove duplicate values
txns.drop_duplicates(subset="client_internal")
batch_upsert_request = {}

for row, instr in txns.iterrows():
    identifiers={}
    if instr["client_internal"] == "cash":
        
        # if the current row is a currency, give it a currency identifier
        identifiers["Currency"] = models.InstrumentIdValue(value=instr["instrument_currency"])
    else:
        
        # if the current row is not a currency, just use the Client Internal ID 
        identifiers["ClientInternal"] = models.InstrumentIdValue(value=instr["client_internal"])

    batch_upsert_request[instr['instrument_desc']] = models.InstrumentDefinition(
        name=instr['instrument_desc'],
        identifiers=identifiers,
        properties=[
            models.ModelProperty(
                key=f"Instrument/{scope}/portfolio_manager_name",
                value=models.PropertyValue(
                    label_value=instr["portfolio_manager_name"]
                )
            )
        ]
    )

# Upsert new instruments to LUSID
instrument_response = InstrumentsApi.upsert_instruments(
    instruments=batch_upsert_request)

# check response was successful

if len(instrument_response.failed) > 0:
        raise AssertionError("Instruments upsert failed. Inspect response for more detail")

# Document section 6: Loading a Transaction File

In [9]:
# Call LUSID to create a property
property_response = PropertyDefinitionsApi.create_property_definition(
definition=models.CreatePropertyDefinitionRequest(
            domain="Transaction",
            scope=scope,
            code="portfolio_manager_name",
            value_required=False,
            display_name="portfolio_manager_name",
            data_type_id=models.ResourceId(
                scope='system',
                code='string'
            )
        )
    )

print(f"Created new property {property_response.key}")

Created new property Transaction/life_of_a_txn-3854-a038-167a-56/portfolio_manager_name


In [10]:
# Upsert transactions
transactions_request=[]
txn_response=[]
for row, txn in txns.iterrows():
    
    if txn["client_internal"] == "cash":
        instrument_identifier={"Instrument/default/Currency": "GBP"}
    else:
        instrument_identifier={"Instrument/default/ClientInternal": txn["client_internal"]}
    
    # build request body
    transactions_request.append(
        models.TransactionRequest(
            transaction_id=txn["txn_id"],
            type=txn["transaction_type"],
            instrument_identifiers=instrument_identifier,
            transaction_date=txn["trade_date"],
            settlement_date=txn["trade_date"],
            units=txn["quantity"],
            transaction_price=models.TransactionPrice(
                price=txn["price"],
                type="Price"
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=txn["net_money"],
                currency=txn["instrument_currency"]
            ),
            properties=
            {
                f"Transaction/{scope}/portfolio_manager_name":
                    models.PerpetualProperty(
                        key=f"Transaction/{scope}/portfolio_manager_name",
                        value=models.PropertyValue(
                            label_value=txn["portfolio_manager_name"]
                        )
                    )
            }
        )
    )
    
    # Make Upsert Transactions call to LUSID
    txn_response.append(
        TransactionPortfoliosApi.upsert_transactions(
            scope=scope,
            code=portfolio_code,
            transactions=transactions_request
        )
    )
    
print(f"{len(txn_response)} transactions upserted")

5 transactions upserted


# Document Section 7: Configuring Transaction Types

In this section we will start by getting the current transaction and side configurations and saving them to a local file so that we reset the original values at the end of the notebook. Setting the transaction configurations will replace any existing configuration so we will begin by saving the existing configuration locally. We recommend that once you have saved this file you also save a backup copy if you accidentally overwrite this file. 

### Warning: We recomend you backup and rename this configuration file

In [11]:
# get current Configuration
default_types = SystemConfigurationApi.list_configuration_transaction_types()

# Save config to .pkl file so that we can reset default values if needed
with open('default_config.pkl', 'wb') as output:
    pickle.dump(default_types, output, pickle.HIGHEST_PROTOCOL)


In [12]:
new_config = models.TransactionSetConfigurationDataRequest(
    transaction_config_requests=[
        models.TransactionConfigurationDataRequest(
            aliases=
            [
                models.TransactionConfigurationTypeAlias(
                    type="Buy",
                    description="A purchase transaction from System X",
                    transaction_class="Basic",
                    transaction_group="default",
                    transaction_roles="LongLonger"
                ),
                models.TransactionConfigurationTypeAlias(
                    type="B",
                    description="A purchase transaction from System X",
                    transaction_class="Basic",
                    transaction_group="alt1",
                    transaction_roles="LongLonger"
                ),
            ],
            movements=
            [
                models.TransactionConfigurationMovementData(
                    movement_types="StockMovement",
                    side="Side1", 
                    direction=1,
                    properties={},
                    mappings=[]
                ),
                models.TransactionConfigurationMovementData(
                    movement_types="CashCommitment",
                    side="Side2",
                    direction=-1,
                    properties={},
                    mappings=[]
                ),
                models.TransactionConfigurationMovementData(
                    movement_types="CashCommitment",
                    side="TradeCommissions",
                    direction=-1,
                    properties={},
                    mappings=[
                        models.TransactionPropertyMappingRequest(
                            property_key=f"Transaction/{scope}/Broker_2",
                            set_to="Commission"
                        )
                    ]
                )
            ],
        properties={}
        ),
        models.TransactionConfigurationDataRequest(
            aliases=
            [
                models.TransactionConfigurationTypeAlias(
                    type="FundsIn",
                    description="Deposit New Funds",
                    transaction_class="CashTransfers",
                    transaction_group="default",
                    transaction_roles="Longer"
                ),
                models.TransactionConfigurationTypeAlias(
                    type="FI",
                    description="Deposit New Funds",
                    transaction_class="CashTransfers",
                    transaction_group="alt1",
                    transaction_roles="Longer"
                ),
            ],
            movements=
            [
                models.TransactionConfigurationMovementData(
                    movement_types="CashAccrual",
                    side="Side1", 
                    direction=1,
                    properties={},
                    mappings=[]
                ),
            ],
        properties={}
        )
    ],
    side_config_requests=[
        models.SideConfigurationDataRequest(
            side="Side1",
            security="Txn:LusidInstrumentId",
            currency="Txn:TradeCurrency",
            rate="Txn:TradeToPortfolioRate",
            units="Txn:Units",
            amount="Txn:TradeAmount"
        ),
        models.SideConfigurationDataRequest(
            side="Side2",
            security="Txn:SettleCcy",
            currency="Txn:SettlementCurrency",
            rate="SettledToPortfolioRate",
            units="Txn:TotalConsideration",
            amount="Txn:TotalConsideration"
        ),
        models.SideConfigurationDataRequest(
            side="TradeCommissions",
            security="Txn:SettleCcy",
            currency="Txn:SettlementCurrency",
            rate="SettledToPortfolioRate",
            units=f"Transaction/{scope}/Comms2",
            amount=f"Transaction/{scope}/Comms2"
        )
    ]
)

# Set our LUSID environment to use the new transaction configuration
response = SystemConfigurationApi.set_configuration_transaction_types(
    types = new_config,
)

# Document section 8: Get Holdings

In [13]:
# Prints the name and a quick summary from a get_holdings() response 
def display_holdings_summary(response):
    # inspect holdings response for today
    hld = [i for i in response.values]
    
    names=[]
    amount=[]
    units=[]
    shks=[]
    
    
    for item in hld:
        
        names.append(item.properties['Instrument/default/Name'].value.label_value)
        amount.append(item.cost.amount)
        units.append(item.units)
        shks.append([item.sub_holding_keys[key].value.label_value for key in item.sub_holding_keys.keys()])
        
    data={
        "names" : names,
        "amount" : amount,
        "units" : units,
        "shks" : shks
    }
    
    summary = pd.DataFrame(data=data)
    return summary

#### Get holdings with todays effective date

In [14]:
holdings_response_today = TransactionPortfoliosApi.get_holdings(
    scope=scope, 
    code=portfolio_code,
    property_keys=["Instrument/default/Name"]
)

summary = display_holdings_summary(holdings_response_today)
summary

,names,amount,units,shks
0,Tesco,308000.0,120000.0,[]
1,Morrisons,230400.0,120000.0,[]
2,Sainsbury's,452000.0,200000.0,[]
3,CCY_GBP,9600.0,9600.0,[]


#### Get holdings effective 1st January 2020 (the trade date of the first transactions)

In [15]:
holdings_response_1st_jan = TransactionPortfoliosApi.get_holdings(
    scope=scope, 
    code=portfolio_code,
    effective_at="2020-01-01T00:00:01.0000000+00:00",
    property_keys=["Instrument/default/Name"]
)

display_holdings_summary(holdings_response_1st_jan)

,names,amount,units,shks
0,Tesco,256000.0,100000.0,[]
1,CCY_GBP,744000.0,744000.0,[]


#### Get holdings effective 1st March (after the trade date of the last transaction)

In [16]:
holdings_response_1st_Mar = TransactionPortfoliosApi.get_holdings(
    scope=scope, 
    code=portfolio_code,
    property_keys=["Instrument/default/Name"],
    effective_at="2020-03-01T00:00:01.0000000+00:00"
)

display_holdings_summary(holdings_response_1st_Mar)

,names,amount,units,shks
0,Tesco,308000.0,120000.0,[]
1,Morrisons,230400.0,120000.0,[]
2,Sainsbury's,452000.0,200000.0,[]
3,CCY_GBP,9600.0,9600.0,[]


# Document section 9: Sub-holding Keys

In order to segregate our new portfolio that includes Sub-holding Keys (SHKs), we will use a different scope

In [17]:
txns_strat = pd.read_csv("data/txn_strategy.csv")

In [18]:
scope_shk = scope + "-stratategy-tagging"
print(scope_shk)

life_of_a_txn-3854-a038-167a-56-stratategy-tagging


#### create the property for `StrategyTag`

In [19]:
# Call LUSID to create our new property
# this property willbe used as our SHK
property_response = PropertyDefinitionsApi.create_property_definition(
    definition=models.CreatePropertyDefinitionRequest(
            domain="Transaction",
            scope=scope_shk,
            code="StrategyTag",
            value_required=False,
            display_name="StrategyTag",
            data_type_id=models.ResourceId(
                scope='system',
                code='string'
            )
        )
    )

print(f"Created new property {property_response.key}")

Created new property Transaction/life_of_a_txn-3854-a038-167a-56-stratategy-tagging/StrategyTag


#### Assign property to portfolio (or derived portfolio) as SHK

In [20]:
# Create portfolio with properties
created_date="2010-01-01T00:00:00.000000+00:00"

# create request body
portfolio_request = models.CreateTransactionPortfolioRequest(
    display_name="Life of a transaction - stratategy tagging",
    code = portfolio_code,
    base_currency="GBP",
    created=created_date,
    sub_holding_keys=[
        f"Transaction/{scope_shk}/StrategyTag"
    ]
)

# Upload new portfolio to LUSID
response = TransactionPortfoliosApi.create_portfolio(
    scope=scope_shk,
    transaction_portfolio=portfolio_request
)

created = response.version.effective_from
print(f"portfolio '{response.id.code}', in scope {scope_shk} created effective from: "
      f"{created.year}/"
      f"{created.month}/"
      f"{created.day}"
     )

portfolio 'UK_EQUITY', in scope life_of_a_txn-3854-a038-167a-56-stratategy-tagging created effective from: 2010/1/1


#### Upload transactions with properties for SHK

In [21]:
# Upsert transactions
transactions_request=[]
txn_response=[]
for row, txn in txns_strat.iterrows():
    
    if txn["client_internal"] == "cash":
        instrument_identifier={"Instrument/default/Currency": "GBP"}
    else:
        instrument_identifier={"Instrument/default/ClientInternal": txn["client_internal"]}
    
    # build request body
    transactions_request.append(
        models.TransactionRequest(
            transaction_id=txn["txn_id"],
            type=txn["transaction_type"],
            instrument_identifiers=instrument_identifier,
            transaction_date=txn["trade_date"],
            settlement_date=txn["trade_date"],
            units=txn["quantity"],
            transaction_price=models.TransactionPrice(
                price=txn["price"],
                type="Price"
            ),
            total_consideration=models.CurrencyAndAmount(
                amount=txn["net_money"],
                currency=txn["instrument_currency"]
            ),
            properties=
            {
                f"Transaction/{scope_shk}/StrategyTag":
                    models.PerpetualProperty(
                        key=f"Transaction/{scope_shk}/StrategyTag",
                        value=models.PropertyValue(
                            label_value=txn["StrategyTag"]
                        )
                    ),
            }
        )
    )
    
    
    # Make Upsert Transactions call to LUSID
    txn_response.append(
        TransactionPortfoliosApi.upsert_transactions(
            scope=scope_shk,
            code=portfolio_code,
            transactions=transactions_request
        )
    )
    
print(f"{len(txn_response)} transactions upserted")

5 transactions upserted


### Get Holdings from new portfolio with SHK

Now when we get holdings, the result will be segregated by any specified Sub-holding Keys that we specified on the portfolio. 

In [22]:
holdings_response_1st_Mar = TransactionPortfoliosApi.get_holdings(
    scope=scope_shk, 
    code=portfolio_code,
    property_keys=[
        "Instrument/default/Name",
    ],
    effective_at="2020-03-01T00:00:01.0000000+00:00"
)

display_holdings_summary(holdings_response_1st_Mar)

,names,amount,units,shks
0,Tesco,256000.0,100000.0,[benchmarkRebalance]
1,Tesco,52000.0,20000.0,[quantitativeSignal]
2,Morrisons,230400.0,120000.0,[benchmarkRebalance]
3,Sainsbury's,452000.0,200000.0,[benchmarkRebalance]
4,CCY_GBP,61600.0,61600.0,[benchmarkRebalance]
5,CCY_GBP,-52000.0,-52000.0,[quantitativeSignal]


In [23]:
holdings_response_1st_jan = TransactionPortfoliosApi.get_holdings(
    scope=scope_shk, 
    code=portfolio_code,
    effective_at="2020-01-01T00:00:01.0000000+00:00",
    property_keys=[
        "Instrument/default/Name",
    ],
)

display_holdings_summary(holdings_response_1st_jan)

,names,amount,units,shks
0,Tesco,256000.0,100000.0,[benchmarkRebalance]
1,CCY_GBP,744000.0,744000.0,[benchmarkRebalance]


# Resetting the original transaction configuration

Now that we have demonstrated how to set custom transaction configurations, we will reset the original configurations using the pickle file we saved earlier. If you have accidentally overwritten this file then replace “default_config.pkl” with your renamed backup file.

In [24]:
# Load config from the .pkl file we saved
with open('default_config.pkl', 'rb') as inpt:
    saved_config=pickle.load(inpt)

# Get Transaction configs and side configs from original config
transaction_config_requests = [
    models.TransactionConfigurationDataRequest(
        aliases=saved_config.transaction_configs[i].aliases,
        movements=saved_config.transaction_configs[i].movements,
        properties=saved_config.transaction_configs[i].properties
    )
    for i in range(len(saved_config.transaction_configs))
]

side_config_requests = [
    models.SideConfigurationDataRequest(
        side=saved_config.side_definitions[i].side,
        security=saved_config.side_definitions[i].security,
        currency=saved_config.side_definitions[i].currency,
        rate=saved_config.side_definitions[i].rate,
        units=saved_config.side_definitions[i].units,
        amount=saved_config.side_definitions[i].amount
    )
    for i in range(len(saved_config.side_definitions))
]

response = SystemConfigurationApi.set_configuration_transaction_types(
    types = models.TransactionSetConfigurationDataRequest(
        transaction_config_requests=transaction_config_requests,
        side_config_requests=side_config_requests
    ),
)